# Init

In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/wliao0504/code/clif/pyCLIF'

## Reload

In [2]:
import pandas as pd
from pathlib import Path
import duckdb

# Import individual table classes
from clifpy.tables.vitals import Vitals
from clifpy.tables.labs import Labs
from clifpy.tables.medication_admin_continuous import MedicationAdminContinuous

from importlib import reload

import yaml

with open('config/config.yaml', 'r') as file:
    config = yaml.safe_load(file)
    
from clifpy.utils import sofa
reload(sofa)

<module 'clifpy.utils.sofa' from '/Users/wliao0504/code/clif/pyCLIF/clifpy/utils/sofa.py'>

# Archive

In [3]:
# # Initialize the tables
# vitals = Vitals.from_file(
#     data_directory=config['tables_path'],
#     filetype='parquet', 
#     timezone='UTC'
# )

# vitals_df = vitals.df

# # Initialize the tables
# labs = Labs.from_file(
#     data_directory=config['tables_path'],
#     filetype='parquet', 
#     timezone='UTC'
# )

# labs_df = labs.df

# # Initialize the tables
# mac = MedicationAdminContinuous.from_file(
#     data_directory=config['tables_path'],
#     filetype='parquet', 
#     timezone='UTC'
# )

# mac_df = mac.df

In [4]:
# Mock DataFrame creation for ids_w_dttm
cohort_df = pd.DataFrame({
    'hospitalization_id': ['23559586', '20626031'],
    'start_time': pd.to_datetime(['2132-12-15 14:29:00+00:00', '2132-12-14 21:00:00+00:00']),
    'end_time': pd.to_datetime(['2137-08-25 14:00:00+00:00', '2137-09-02 09:00:00+00:00'])
})

# Using Wide

In [5]:
from clifpy.utils import create_wide_dataset
from clifpy.clif_orchestrator import ClifOrchestrator

In [6]:
co = ClifOrchestrator(
    data_directory='/Users/wliao0504/code/clif/pyCLIF/clifpy/data/clif_demo',
    filetype='parquet',
    timezone='UTC',
    output_directory= None)

co.load_table('vitals')
co.load_table('labs')
co.load_table('patient_assessments')
co.load_table('medication_admin_continuous')
co.load_table('respiratory_support')

Using directly provided parameters
ClifOrchestrator initialized.
Using directly provided parameters
Loading clif_vitals.parquet
Data loaded successfully from clif_vitals.parquet
recorded_dttm: Already in your timezone (UTC), no conversion needed.
Using directly provided parameters
Loading clif_labs.parquet
Data loaded successfully from clif_labs.parquet
lab_order_dttm: Already in your timezone (UTC), no conversion needed.
lab_collect_dttm: Already in your timezone (UTC), no conversion needed.
lab_result_dttm: Already in your timezone (UTC), no conversion needed.
Using directly provided parameters
Loading clif_patient_assessments.parquet
Data loaded successfully from clif_patient_assessments.parquet
recorded_dttm: Already in your timezone (UTC), no conversion needed.
Using directly provided parameters
Loading clif_medication_admin_continuous.parquet
Data loaded successfully from clif_medication_admin_continuous.parquet
admin_dttm: Already in your timezone (UTC), no conversion needed.
Us

In [7]:
cohort_df = pd.DataFrame({
    'hospitalization_id': ['23559586', '20626031'],
    'start_time': pd.to_datetime(['2137-01-01 14:29:00+00:00', '2132-12-14 21:00:00+00:00']),
    'end_time': pd.to_datetime(['2137-08-25 14:00:00+00:00', '2132-12-15 09:00:00+00:00'])
})

required_sofa_categories_by_table = {
    'labs': ['creatinine','platelet_count','po2_arterial','bilirubin_total'],
    'vitals': ['map','spo2', 'weight_kg'],
    'patient_assessments': ['gcs_total'],
    "medication_admin_continuous": [
        "norepinephrine","epinephrine","phenylephrine","vasopressin", "dopamine","angiotensin",
        "dobutamine","milrinone"
        ],
    'respiratory_support': [
        'device_category','device_name','mode_name','mode_category','peep_set','fio2_set','lpm_set',
        'resp_rate_set','tracheostomy', 'resp_rate_obs','tidal_volume_set'
    ] 
}

wide_df = co.create_wide_dataset(
    tables_to_load=[
        'vitals', 'labs', 'patient_assessments', 'medication_admin_continuous', 'respiratory_support'
        ],
    category_filters=required_sofa_categories_by_table,
    # sample=True,  # Use 20 random hospitalizations
    cohort_df=cohort_df,
    batch_size=-1
)

Loading patient table...
Using directly provided parameters
Loading clif_patient.parquet
Data loaded successfully from clif_patient.parquet
death_dttm: Already in your timezone (UTC), no conversion needed.
Loading hospitalization table...
Using directly provided parameters
Loading clif_hospitalization.parquet
Data loaded successfully from clif_hospitalization.parquet
admission_dttm: Already in your timezone (UTC), no conversion needed.
discharge_dttm: Already in your timezone (UTC), no conversion needed.
Loading adt table...
Using directly provided parameters
Loading clif_adt.parquet
Data loaded successfully from clif_adt.parquet
in_dttm: Already in your timezone (UTC), no conversion needed.
out_dttm: Already in your timezone (UTC), no conversion needed.
Starting wide dataset creation...
Using cohort_df with time windows for 2 hospitalizations
Using 2 hospitalization IDs from cohort_df

Loading and filtering base tables...
  ADT time filtering: 11 → 6 records
Base tables filtered - Hos

In [8]:
device_rank_dict = {
    'IMV': 1,
    'NIPPV': 2,
    'CPAP': 3,
    'High Flow NC': 4,
    'Face Mask': 5,
    'Trach Collar': 6,
    'Nasal Cannula': 7,
    'Other': 8,
    'Room Air': 9
}

# create a mappping df
device_rank_mapping = pd.DataFrame({
    'device_category': device_rank_dict.keys(),
    'device_rank': device_rank_dict.values()
})

# q = """
# FROM wide_df
# LEFT JOIN device_rank_mapping USING (device_category)
# SELECT *
#     , device_rank_min: MIN(device_rank)
#     , fio2_max: MAX(fio2_set)
# GROUP BY hospitalization_id
# """
# device_ranked = duckdb.sql(q).df()

In [9]:
required_sofa_categories = [item for sublist in required_sofa_categories_by_table.values() for item in sublist] 

In [ ]:
def generate_extremal_values_by_id(
    wide_df: pd.DataFrame,
    extremal_type: str # worst or latest
) -> pd.DataFrame:
    pass


In [ ]:
q = f"""
FROM wide_df
LEFT JOIN device_rank_mapping USING (device_category)
SELECT hospitalization_id
    -- , MAX(COLUMNS({required_sofa_categories})) AS 'max_\\0'
    , any_value(COLUMNS({required_sofa_categories}) ORDER BY event_time DESC) -- AS 'latest_\\0'
    , device_rank_min: MIN(device_rank)
GROUP BY hospitalization_id
"""
latest_by_id = duckdb.sql(q).df()

In [16]:
max_items = required_sofa_categories_by_table['medication_admin_continuous'] \
    + ['fio2_set', 'creatinine', 'bilirubin_total']
min_items = ['map', 'spo2', 'po2_arterial', 'platelet_count', 'gcs_total']

In [17]:
q = f"""
FROM wide_df
LEFT JOIN device_rank_mapping USING (device_category)
SELECT hospitalization_id
    , MAX(COLUMNS({max_items})) -- AS 'max_\\0'
    , MIN(COLUMNS({min_items})) -- AS 'min_\\0'
    , device_rank_min: MIN(device_rank)
GROUP BY hospitalization_id
"""
worst_by_id = duckdb.sql(q).df()

In [19]:
extremals_by_id = 'worst_by_id'
q = f"""
FROM {extremals_by_id} df
LEFT JOIN device_rank_mapping m on df.device_rank_min = m.device_rank
SELECT hospitalization_id
    , p_f: po2_arterial / fio2_set
    , sofa_cv_97: CASE WHEN dopamine > 15 OR epinephrine > 0.1 OR norepinephrine > 0.1 THEN 4
        WHEN dopamine > 5 OR epinephrine <= 0.1 OR norepinephrine <= 0.1 THEN 3
        WHEN dopamine <= 5 OR dobutamine > 0 THEN 2
        WHEN map < 70 THEN 1
        WHEN map >= 70 THEN 0 END
    , sofa_coag: CASE WHEN platelet_count < 20 THEN 4
        WHEN platelet_count < 50 THEN 3
        WHEN platelet_count < 100 THEN 2
        WHEN platelet_count < 150 THEN 1
        WHEN platelet_count >= 150 THEN 0 END 
    , sofa_liver: CASE WHEN bilirubin_total >= 12 THEN 4
        WHEN bilirubin_total < 12 AND bilirubin_total >= 6 THEN 3
        WHEN bilirubin_total < 6 AND bilirubin_total >= 2 THEN 2
        WHEN bilirubin_total < 2 AND bilirubin_total >= 1.2 THEN 1
        WHEN bilirubin_total < 1.2 THEN 0 END
    , sofa_resp: CASE WHEN p_f < 100 AND m.device_category IN ('imv', 'nippv', 'cpap') THEN 4 
        WHEN p_f >= 100 and p_f < 200 AND m.device_category IN ('imv', 'nippv', 'cpap') THEN 3
        WHEN p_f >= 200 AND p_f < 300 THEN 2
        WHEN p_f >= 300 AND p_f < 400 THEN 1
        WHEN p_f >= 400 THEN 0 END
    , sofa_cns: CASE WHEN gcs_total < 6 THEN 4
        WHEN gcs_total >= 6 AND gcs_total <= 9 THEN 3
        WHEN gcs_total >= 10 AND gcs_total <= 12 THEN 2
        WHEN gcs_total >= 13 AND gcs_total <= 14 THEN 1
        WHEN gcs_total == 15 THEN 0 END
    , sofa_renal: CASE WHEN creatinine >= 5 THEN 4
        WHEN creatinine < 5 AND creatinine >= 3.5 THEN 3
        WHEN creatinine < 3.5 AND creatinine >= 2 THEN 2
        WHEN creatinine < 2 AND creatinine >= 1.2 THEN 1
        WHEN creatinine < 1.2 THEN 0 END
    , sofa_total: sofa_cv_97 + sofa_coag + sofa_liver + sofa_resp + sofa_renal + sofa_cns
"""
df = duckdb.sql(q).df()